# BASE 1719

Information:
* Periodo: 2017-2019
* Objetivo: predicción de pobreza

## 1. Importar datos y librerias

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )

In [ ]:
import pandas as  pd
import numpy as np
import pickle
import joblib

In [ ]:
import variables_nombres as vn

In [ ]:
path = r'..\..\..\input\vulnerability_line\base_17_19_vl.csv'
data_original = pd.read_csv( path )

In [ ]:
data_original.shape

### 2. Preprocesar datos

In [ ]:
# Imputación de valores del gasto del percentil 1% superior con valores del percentil 99

percentil_superior = 0.01
perc = data_original[ 'gasto_deflactado' ].quantile( 1 - percentil_superior )
data.loc[ data_original [ 'gasto_deflactado' ] > perc, 'gasto_deflactado' ] = perc

In [ ]:
next_vars = [ 'gasto_deflactado', 'regiones' ]
data      = data_original[ next_vars ]

### 3. Importar modelos entrenados y predecir

In [ ]:
rf_optimal_model_o = joblib.load( r'..\..\output\ejecucion_1\models\base_17_19\base_17_19_lg_model_ojoblib' )

y_rf_pred_class_o  = rf_optimal_model_o.predict( data )
y_rf_pred_prob_o   = rf_optimal_model_o.predict_proba( data )[ :, 1 ]

### 4. Identificar hogares vulnerables

In [ ]:
data_completa = data_original.copy( deep = True )

data_completa[ 'class_pobreza' ] = y_rf_pred_class_o
data_completa[ 'prob_pobreza' ]  = y_rf_pred_prob_o

Se considera distintas amplitudes de caliper: 
* 1: +-   2
* 2: +-   1
* 3: +- 0.5

In [ ]:
limite_min_1 = 0.070
limite_max_1 = 0.110

limite_min_2 = 0.080
limite_max_2 = 0.100

limite_min_3 = 0.085
limite_max_3 = 0.095

hogares_vulnerables_1 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_1 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_1 ) ]
hogares_vulnerables_2 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_2 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_2 ) ]
hogares_vulnerables_3 = data_completa[ ( data_completa[ 'prob_pobreza' ] >= limite_min_3 ) & data_completa[ ( data_completa[ 'prob_pobreza' ] <= limite_max_3 ) ]

### 5. Calcular la línea de vulnerabilidad

In [ ]:
linea_vul_1 = hogares_vulnerables_1[ 'gasto_deflactado' ].mean()
linea_vul_1

In [ ]:
linea_vul_2 = hogares_vulnerables_2[ 'gasto_deflactado' ].mean()
linea_vul_2

In [ ]:
linea_vul_2 = hogares_vulnerables_2[ 'gasto_deflactado' ].mean()
linea_vul_2